# Kalman Filter Algorithm

The kalman filter algorithm is a gaussian filter that attempts to estimate the mean $\mu$ and variance $\sigma^2$ of the state using process and measurement information. The algorithm is defined below

$
\begin{align}
&\textbf{Prediction} \\
&x = Fx \\
&P = FPF^T + Q \\
&\textbf{Update} \\
&y = z - Hx \\
&S = HPH^T + R \\
&K = PH^TS^{-1} \\
&x = x + Ky \\
&P = (I - KH)P \\
\end{align} \\
$

- **x**: state vector
- **F**: state transition matrix
- **P**: state covariance matrix
- **Q**: process covariance matrix
- **z**: measurement vector
- **H**: map state to measurement space
- **R**: measurement covariance matrix
- **S**: Combination of state covariance and measurement covariance
- **K**: Kalman filter gain

This algorithm is broken into two steps, prediction and measurement update.

## Prediction

In the prediction step we predict what the next state is going to be using a linear model, **F**, and a process covariance matrix, **Q**, that captures model uncertainty. In the module the problem we are trying to solve is estimating the 2D position and velocity of a moving object.

For this problem we assume that the velocity is constant so the kinematic equations are the following

$
\begin{align}
&p_x = p_x + v_x \Delta t \\
&p_y = p_y + v_y \Delta t \\
&v_x = v_x \\
&v_y = v_y \\
\end{align}
\rightarrow
\begin{bmatrix}
p_x \\
p_y \\
v_x \\
v_y \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & \Delta t & 0 \\
0 & 1 & 0 & \Delta t \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
p_x \\
p_y \\
v_x \\
v_y \\
\end{bmatrix}
+
\begin{bmatrix}
\nu_{p_x} \\
\nu_{p_y} \\
\nu_{v_x} \\
\nu_{v_y} \\
\end{bmatrix}
$

The next step is to update the state covariance using the state transition matrix defined above and the process covariance matrix. This models the stochastic part of the state transition matrix. First lets revist the kinematic equations with the acceleration term included.

$
\begin{align}
&p_x = p_x + v_x \Delta t + a_x \frac{\Delta t^2}{2} \\
&p_y = p_y + v_y \Delta t + a_y \frac{\Delta t^2}{2} \\
&v_x = v_x + a_x \Delta t \\
&v_y = v_y + a_y \Delta t \\
\end{align}
$

Now lets make the assumption that the acceleration terms are random variables with zero mean and variance, $\sigma_{ax}^2$ and $\sigma_{ay}^2$.

$
\begin{align}
&p_x = p_x + v_x \Delta t + a_x \frac{\Delta t^2}{2} \\
&p_y = p_y + v_y \Delta t + a_y \frac{\Delta t^2}{2} \\
&v_x = v_x + a_x \Delta t \\
&v_y = v_y + a_y \Delta t \\
\end{align}
\rightarrow
\begin{bmatrix}
p_x \\
p_y \\
v_x \\
v_y \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & \Delta t & 0 \\
0 & 1 & 0 & \Delta t \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
p_x \\
p_y \\
v_x \\
v_y \\
\end{bmatrix}
+
\begin{bmatrix}
\frac{\Delta t^2}{2} & 0 \\
0 & \frac{\Delta t^2}{2} \\
\Delta t & 0 \\
0 & \Delta t \\
\end{bmatrix}
\begin{bmatrix}
a_x \\
a_y \\
\end{bmatrix}
$

The covariance can be computed using this formula

$
Q =
E(\nu\nu^T) =
\begin{bmatrix}
\frac{\Delta t^2}{2} & 0 \\
0 & \frac{\Delta t^2}{2} \\
\Delta t & 0 \\
0 & \Delta t \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{ax}^2 & \sigma_{axy} \\
\sigma_{axy} & \sigma_{ay}^2 \\
\end{bmatrix}
\begin{bmatrix}
\frac{\Delta t^2}{2} & 0 \\
0 & \frac{\Delta t^2}{2} \\
\Delta t & 0 \\
0 & \Delta t \\
\end{bmatrix}^T - 
\begin{bmatrix}
0 & 0 \\
0 & 0 \\
\end{bmatrix}
=
\left[\begin{matrix}\frac{\sigma_{ax}^2}{4} {\Delta t}^{4} & \frac{\sigma_{axy}}{4} {\Delta t}^{4} & \frac{\sigma_{ax}^2}{2} {\Delta t}^{3} & \frac{\sigma_{axy}}{2} {\Delta t}^{3}\\\frac{\sigma_{axy}}{4} {\Delta t}^{4} & \frac{\sigma_{ay}^2}{4} {\Delta t}^{4} & \frac{\sigma_{axy}}{2} {\Delta t}^{3} & \frac{\sigma_{ay}^2}{2} {\Delta t}^{3}\\\frac{\sigma_{ax}^2}{2} {\Delta t}^{3} & \frac{\sigma_{axy}}{2} {\Delta t}^{3} & \sigma_{ax}^2 {\Delta t}^{2} & \sigma_{axy} {\Delta t}^{2}\\\frac{\sigma_{axy}}{2} {\Delta t}^{3} & \frac{\sigma_{ay}^2}{2} {\Delta t}^{3} & \sigma_{axy} {\Delta t}^{2} & \sigma_{ay}^2 {\Delta t}^{2}\end{matrix}\right]
$

In this case we are going to assume that the acceleration in x and y is zero so the covariance term, $\sigma_{axy}$ is equal to zero. This results in the following process covariance matrix

$
Q = \left[\begin{matrix}\frac{\sigma_{ax}^2}{4} {\Delta t}^{4} & 0 & \frac{\sigma_{ax}^2}{2} {\Delta t}^{3} & 0\\0 & \frac{\sigma_{ay}^2}{4} {\Delta t}^{4} & 0 & \frac{\sigma_{ay}^2}{2} {\Delta t}^{3}\\\frac{\sigma_{ax}^2}{2} {\Delta t}^{3} & 0 & \sigma_{ax}^2 {\Delta t}^{2} & 0\\0 & \frac{\sigma_{ay}^2}{2} {\Delta t}^{3} & 0 & \sigma_{ay}^2 {\Delta t}^{2}\end{matrix}\right]
$

To summarize the prediction step, we compute the state and state covariance matrix using the equations below

$
\begin{align}
&x = Fx \\
&P = FPF^T + Q \\
\end{align}
$

**Questions**
- How do we select $\sigma_{ax}^2$, and $\sigma_{ay}^2$? 

## Measurement Update

In the measurement step we estimate the state and state covariance matrix using a measurement. The first few steps are done in measurement space and we then convert back to state space to compute the new state estimate and state covariance matrix estimate.

The measurement vector, **z**, is receieved from the sensor. In the case of lidar this is the x and y position of the object, [px, py]. The next step we need to do is convert our current state estimation to measurement space to find the error. The state space has dimension 4, [px, py, vx, vy] and we need to project this to the measurement space which is two dimensions. The corresponding matrix that does this is

$
H = 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
\end{bmatrix}
$

Using the state space to measurement space transformation matrix we can use this to complete the measurement step using the equations above.

$
\begin{align}
&y = z - Hx \\
&S = HPH^T + R \\
&K = PH^TS^{-1} \\
&x = x + Ky \\
&P = (I - KH)P \\
\end{align}
$

To incorporate measurement uncertainty we use the sensor covariance matrix, **R**. This is normally provided by the sensor manufacturer.

# Extended Kalman Filter

Alright up to this point we have had a linear mapping from measurement space to state space. The extended kalman filter can be used when the measurement space to state space is nonlinear. The example used in this module is radar. Radar measures radial position, heading, and radial velocity.

$
z = 
\begin{bmatrix}
\rho \\
\phi \\
\dot{\rho} \\
\end{bmatrix}
$

We can write these parameters in terms of the state space parameters

$
\begin{bmatrix}
\rho \\
\phi \\
\dot{\rho} \\
\end{bmatrix} =
\begin{bmatrix}
\sqrt{p_{x}^2 + p_{y}^2} \\
tan^{-1}\left(\frac{p_{y}}{p_{x}}\right) \\
\frac{p_x v_x + p_y v_y}{\sqrt{p_{x}^2 + p_{y}^2}} \\
\end{bmatrix}
$